<a href="https://colab.research.google.com/github/mk2436/CYBERSECURITY-PREDICTION-MODEL/blob/main/models/RandomForest_097_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score
import xgboost as xgb
import matplotlib.pyplot as plt
from google.colab import files


file_name = 'drive/MyDrive/AI/cybersecurity_attacks.csv'
data = pd.read_csv(file_name)
# Display the first few rows to understand the dataset
print("Dataset Preview:")
print(data.head())


Dataset Preview:
             Timestamp Source IP Address Destination IP Address  Source Port  \
0  2023-05-30 06:33:58     103.216.15.12           84.9.164.252        31225   
1  2020-08-26 07:08:30    78.199.217.198         66.191.137.154        17245   
2  2022-11-13 08:23:25      63.79.210.48          198.219.82.17        16811   
3  2023-07-02 10:38:46     163.42.196.10        101.228.192.255        20018   
4  2023-07-16 13:11:07     71.166.185.76        189.243.174.238         6131   

   Destination Port Protocol  Packet Length Packet Type Traffic Type  \
0             17616     ICMP            503        Data         HTTP   
1             48166     ICMP           1174        Data         HTTP   
2             53600      UDP            306     Control         HTTP   
3             32534      UDP            385        Data         HTTP   
4             26646      TCP           1462        Data          DNS   

                                        Payload Data  ... Action Take

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data.shape
data.describe()

Source Port  Destination Port  Packet Length  Anomaly Scores
count  40000.000000      40000.000000   40000.000000    40000.000000
mean   32970.356450      33150.868650     781.452725       50.113473
std    18560.425604      18574.668842     416.044192       28.853598
min     1027.000000       1024.000000      64.000000        0.000000
25%    16850.750000      17094.750000     420.000000       25.150000
50%    32856.000000      33004.500000     782.000000       50.345000
75%    48928.250000      49287.000000    1143.000000       75.030000
max    65530.000000      65535.000000    1500.000000      100.000000

Data Preprocessing

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Timestamp               40000 non-null  object 
 1   Source IP Address       40000 non-null  object 
 2   Destination IP Address  40000 non-null  object 
 3   Source Port             40000 non-null  int64  
 4   Destination Port        40000 non-null  int64  
 5   Protocol                40000 non-null  object 
 6   Packet Length           40000 non-null  int64  
 7   Packet Type             40000 non-null  object 
 8   Traffic Type            40000 non-null  object 
 9   Payload Data            40000 non-null  object 
 10  Malware Indicators      20000 non-null  object 
 11  Anomaly Scores          40000 non-null  float64
 12  Alerts/Warnings         19933 non-null  object 
 13  Attack Type             40000 non-null  object 
 14  Attack Signature        40000 non-null

In [6]:
# Fill missing values for categorical columns with 'Unknown'
categorical_cols = data.select_dtypes(include=['object']).columns
data[categorical_cols] = data[categorical_cols].fillna('Unknown')

# Fill missing values for numerical columns with the median
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].median())


Encode Categorical Variables

In [7]:
# Label Encoding for categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Save the encoders for later use


Feature and Target Selection

In [8]:
# Separate features (X) and target variable (y)
X = data.drop(columns=['Attack Type'])  # Replace 'Attack Type' with the correct target column
y = data['Attack Type']

# Encode the target variable if it is categorical
y = LabelEncoder().fit_transform(y)


Split the Dataset

In [9]:
# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training Set Size: {X_train.shape}")
print(f"Test Set Size: {X_test.shape}")


Training Set Size: (32000, 24)
Test Set Size: (8000, 24)


**Low Accuracy without bias**

In [ ]:
# Import necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
from imblearn.over_sampling import SMOTE

# Step 1: Balance the Dataset (if necessary)
print("Class Distribution Before Balancing:")
print(pd.Series(y).value_counts())

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Class Distribution After Balancing:")
print(pd.Series(y_resampled).value_counts())

# Step 2: Feature Selection using Feature Importance
model_temp = RandomForestClassifier(random_state=42)
model_temp.fit(X_resampled, y_resampled)

# Extract important features
feature_importances = model_temp.feature_importances_
important_features = [X.columns[i] for i in range(len(feature_importances)) if feature_importances[i] > 0.01]

# Filter the dataset
X_filtered = X_resampled[important_features]

# Step 3: Hyperparameter Tuning
param_grid = {
    'n_estimators': [50, 100, 150],  # Lower number of trees
    'max_depth': [5, 10, 15, 20],   # Shallow to medium tree depths
    'min_samples_split': [2, 4, 6], # Stricter split criteria
    'min_samples_leaf': [2, 4, 8],  # Prevent deep trees
}


grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_filtered, y_resampled)

# Best Parameters
best_params = grid_search.best_params_
print("Best Parameters Found:", best_params)

# Step 4: Train with Tuned Parameters
final_model = RandomForestClassifier(**best_params, random_state=42)
final_model.fit(X_filtered, y_resampled)

# Evaluate on Test Set
X_test_filtered = X_test[important_features]
y_pred = final_model.predict(X_test_filtered)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Improved Accuracy: {accuracy:.2f}")
print(f"Improved F1 Score: {f1:.2f}")
print(f"Improved Recall: {recall:.2f}")


Class Distribution Before Balancing:
0    13428
2    13307
1    13265
Name: count, dtype: int64
Class Distribution After Balancing:
2    13428
0    13428
1    13428
Name: count, dtype: int64
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters Found: {'max_depth': 20, 'min_samples_leaf': 8, 'min_samples_split': 2, 'n_estimators': 50}
Improved Accuracy: 0.94
Improved F1 Score: 0.94
Improved Recall: 0.94


**Pre-Procssing**


1.   Dropping the "Geo-location Data" feature
2.   Replacing NULL values with "Unkown", in an attempt to retain the feature



In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = 'drive/MyDrive/AI/cybersecurity_attacks.csv'  # Adjust the path as necessary
data = pd.read_csv(file_path)

# Drop the "Geo-location Data" column
data.drop(columns=["Geo-location Data"], inplace=True)

# Fill missing values for categorical columns with 'Unknown'
categorical_cols = data.select_dtypes(include=['object']).columns
data[categorical_cols] = data[categorical_cols].fillna('Unknown')

# Fill missing values for numerical columns with the median
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].median())

# Label Encoding for categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Save the encoders for later use

# Separate features (X) and target variable (y)
target_column = "Attack Type"  # Replace with the actual target column name
X = data.drop(columns=[target_column])
y = data[target_column]

# Encode the target variable
y = LabelEncoder().fit_transform(y)

# Shapes of X and y
print(X.shape, y.shape)


(40000, 23) (40000,)



1.   Balancing the classes using SMOTE
2.   Feature Selection
1.   Hyperparameter Tunning
2.   Train and Testing





In [12]:
# Import necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
from imblearn.over_sampling import SMOTE

# Step 1: Balance the Dataset (if necessary)
print("Class Distribution Before Balancing:")
print(pd.Series(y).value_counts())

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Class Distribution After Balancing:")
print(pd.Series(y_resampled).value_counts())

# Step 2: Feature Selection using Feature Importance
model_temp = RandomForestClassifier(random_state=42)
model_temp.fit(X_resampled, y_resampled)

# Extract important features
feature_importances = model_temp.feature_importances_
important_features = [X.columns[i] for i in range(len(feature_importances)) if feature_importances[i] > 0.01]

# Filter the dataset
X_filtered = X_resampled[important_features]

# Step 3: Hyperparameter Tuning
param_grid = {
    'n_estimators': [50, 100, 150],  # Lower number of trees
    'max_depth': [5, 10, 15, 20],   # Shallow to medium tree depths
    'min_samples_split': [2, 4, 6], # Stricter split criteria
    'min_samples_leaf': [2, 4, 8],  # Prevent deep trees
}


grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_filtered, y_resampled)

# Best Parameters
best_params = grid_search.best_params_
print("Best Parameters Found:", best_params)

# Step 4: Train with Tuned Parameters
final_model = RandomForestClassifier(**best_params, random_state=42)
final_model.fit(X_filtered, y_resampled)

# Evaluate on Test Set
X_test_filtered = X_test[important_features]
y_pred = final_model.predict(X_test_filtered)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Improved Accuracy: {accuracy:.2f}")
print(f"Improved F1 Score: {f1:.2f}")
print(f"Improved Recall: {recall:.2f}")


Class Distribution Before Balancing:
0    13428
2    13307
1    13265
Name: count, dtype: int64
Class Distribution After Balancing:
2    13428
0    13428
1    13428
Name: count, dtype: int64
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters Found: {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 2, 'n_estimators': 100}
Improved Accuracy: 0.64
Improved F1 Score: 0.64
Improved Recall: 0.64


**Improved Accuracy by manipulating Hyper Tunning**

In [13]:
# Import necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
from imblearn.over_sampling import SMOTE

# Step 1: Balance the Dataset (if necessary)
print("Class Distribution Before Balancing:")
print(pd.Series(y).value_counts())

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Class Distribution After Balancing:")
print(pd.Series(y_resampled).value_counts())

# Step 2: Feature Selection using Feature Importance
model_temp = RandomForestClassifier(random_state=42)
model_temp.fit(X_resampled, y_resampled)

# Extract important features
feature_importances = model_temp.feature_importances_
important_features = [X.columns[i] for i in range(len(feature_importances)) if feature_importances[i] > 0.01]

# Filter the dataset
X_filtered = X_resampled[important_features]

# Step 3: Hyperparameter Tuning
param_grid = {
    'n_estimators': [50, 100, 150],  # Lower number of trees
    'max_depth': [5, 10, 15, 20, 25],   # Shallow to medium tree depths
    'min_samples_split': [2, 4, 6], # Stricter split criteria
    'min_samples_leaf': [2, 4, 8],  # Prevent deep trees
}


grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_filtered, y_resampled)

# Best Parameters
best_params = grid_search.best_params_
print("Best Parameters Found:", best_params)

# Step 4: Train with Tuned Parameters
final_model = RandomForestClassifier(**best_params, random_state=42)
final_model.fit(X_filtered, y_resampled)

# Evaluate on Test Set
X_test_filtered = X_test[important_features]
y_pred = final_model.predict(X_test_filtered)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Improved Accuracy: {accuracy:.2f}")
print(f"Improved F1 Score: {f1:.2f}")
print(f"Improved Recall: {recall:.2f}")


Class Distribution Before Balancing:
0    13428
2    13307
1    13265
Name: count, dtype: int64
Class Distribution After Balancing:
2    13428
0    13428
1    13428
Name: count, dtype: int64
Fitting 3 folds for each of 135 candidates, totalling 405 fits
Best Parameters Found: {'max_depth': 25, 'min_samples_leaf': 8, 'min_samples_split': 2, 'n_estimators': 50}
Improved Accuracy: 0.97
Improved F1 Score: 0.97
Improved Recall: 0.97
